In [ ]:
import os
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.linalg import SparseVector
from scipy.sparse import load_npz
from typing import Tuple
import utils


os.environ['PYSPARK_PYTHON'] = 'C:\ProgramData\mambaforge\envs\ML-base\python.exe'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:\ProgramData\mambaforge\envs\ML-base\Scripts\ipython.exe'

spark = SparkSession.builder.appName("MyApp").getOrCreate()
sc=spark.sparkContext
spark

In [ ]:
df_id=pd.read_parquet("results/bug_pair_ids.parquet")
df_t=pd.read_parquet("results/test_df.parquet")["df_t"]
threshold=0.8

In [ ]:
#toy_df=pd.DataFrame([["Hi how are you John?", 100], ["Hi how are you John my bro?", 101], ["Bro, what girls?", 102]], columns=["text", "id"])

In [ ]:
#toy_df

In [ ]:
""" sparse_repr, _=utils.compute_sparse_repr(toy_df["text"])
pairs, num_of_pairs=utils.compute_cosine_similarity(sparse_repr, toy_df["id"], 0.0)
pairs """

In [ ]:
sparse_repr=load_npz("results/bug_pair_ids.npz")

In [ ]:
doc_feq_sort_idx=np.argsort(df_t.values)[::-1]
#doc_feq_sort_idx_sc=sc.broadcast(doc_feq_sort_idx)

In [ ]:
def csr_to_sparse_vector(row):
    #id non zero dati da
    dtype_tuple=np.dtype([('integer', int), ('float', float)])
    tmp=np.empty(row.indices.shape[0], dtype=dtype_tuple)
    j=0
    for i in doc_feq_sort_idx:
        if row[0, i] !=0.0:
            tmp[j]=(i, row[0, i])
            j+=1
    return SparseVector(row.shape[1], tmp.tolist())

docs_sparse_forSpark = [csr_to_sparse_vector(sparse_repr.getrow(i)) for i in range(sparse_repr.shape[0])]
doc_ids = df_id["_id"].reset_index(drop=True)

In [ ]:
d_star=sparse_repr.max(axis=0).toarray().reshape(-1)

In [ ]:
d_star_sc=sc.broadcast(d_star)
rdd_forMap=sc.parallelize([(doc_ids[i], docs_sparse_forSpark[i]) for i in range(sparse_repr.shape[0])])

In [ ]:
def b_d(sparse_repr):
    cum_sum=0
    res=0
    for i in range(sparse_repr.indices.shape[0]):
        #if t_idx==18656:
        #    print("CIAO")
        #if sparse_repr[t_idx]!=0.0:
        j=int(i)
        res=j
        t=int(sparse_repr.indices[j])
        mult_val=sparse_repr[t]*d_star_sc.value[t]
        cum_sum+=mult_val
        if cum_sum>=threshold:
            res=j-1
            break
        
    return res


def my_map(elem):
    #result=[]
    doc_id=elem[0]
    sparse_repr=elem[1]

    bound=b_d(sparse_repr)
    result=[(t, (doc_id, sparse_repr)) for t in sparse_repr.indices[bound+1:]] #if sparse_repr[t]!=0.0]
    
    """ for i, t_idx in enumerate(sorted_index):
        if i>bound:
            result.append( (t_idx, (doc_id, sparse_repr)) ) """
    return result

rdd_forReduce=rdd_forMap.flatMap(my_map)


In [ ]:
""" from itertools import chain

def flatmap(func, iterable):
    return list(chain.from_iterable(map(func, iterable)))

iter=flatmap(my_map, rdd_forMap)

iter_map=map(my_map, rdd_forMap) """
#iter

In [ ]:
#tmp=list(iter_map)

In [ ]:
#len(set([i[0] for i in tmp[0]]).intersection(set([i[0] for i in tmp[0]])))

In [ ]:
#[i[0] for i in tmp[0]]

In [ ]:
#max(set([i[0] for i in tmp[0]]).intersection(set([i[0] for i in tmp[0]])))

In [ ]:
#max(set(docs_sparse_forSpark[0].indices).intersection(set(docs_sparse_forSpark[1].indices))) #18656

In [ ]:
def max_of_intersection(list1, list2):
    max=0
    i = 0
    j = 0
    while i < len(list1) and j < len(list2):
        elem1=list1[i]
        elem2=list2[j]

        if elem1 == elem2:
            if elem1>max:
                max=elem1
            i += 1
            j += 1
        elif elem1 < elem2:
            i += 1
        else:
            j += 1

    return max


def my_reduce(elem):
    result=[]
    #For directly prune the symmetric pairs 
    pairs_dict={}
    key=elem[0]
    values=elem[1]
    for id1, d1 in values:
        for id2, d2 in values:
            if id1!=id2 and (not pairs_dict.get((id2, id1), False)) and key==max_of_intersection(d1.indices, d2.indices):
                sim=round(d1.dot(d2), 4) #because vector are already normalized
                if sim>=threshold:
                    pairs_dict[(id1, id2)]=True
                    result.append((id1, id2, sim))

    return result

result_pairs=rdd_forReduce.groupByKey().flatMap(my_reduce)

In [ ]:
result_pairs.collect()

In [ ]:
#gb=sc.parallelize(iter).groupByKey().mapValues(list).collect()

In [ ]:
#print([i[0] for i in tmp[0]])

In [ ]:
#flatmap(my_reduce, gb)

In [ ]:
#risultato=sc.parallelize(iter).groupByKey().mapValues(list).flatMap(my_reduce).collect()
#print(risultato)

In [ ]:
spark.stop()